In [1]:
import pandas as pd

# Load your dataset
df = pd.read_excel("51journals_UTD_FT_openalexWOSdoi_utdft.xlsx")

# Split affiliations
def split_affils(val):
    if pd.isna(val):
        return []
    return [x.strip() for x in val.split(";") if x.strip()]

df["affil_list"] = df["affiliations"].apply(split_affils)

# Split authors
def split_authors(val):
    if pd.isna(val):
        return []
    return [x.strip() for x in val.split(";") if x.strip()]

df["author_list"] = df["authors"].apply(split_authors)

# ----------------------------------------------------
# LONG FORMAT: Paper × University
# ----------------------------------------------------
rows_univ = []

for idx, row in df.iterrows():
    unique_unis = set(row["affil_list"])  # unique universities per paper
    
    for uni in unique_unis:
        rows_univ.append({
            "paper_id": int(idx),
            "title": row["title"],
            "year": int(row["year"]),
            "journal": row["journalFullName"],
            "university": uni,
            "disciplineAbbr": row["disciplineAbbr"],
            "utd24": int(row["utd24"]),
            "ft50": int(row["ft50"])
        })

df_univ = pd.DataFrame(rows_univ)

# ================================
# APPLY STATA CLEANING RULES HERE
# ================================

# Trim internal multiple spaces
df_univ["university"] = df_univ["university"].str.replace(r"\s+", " ", regex=True).str.strip()

# ----------------------------------------------------
# REPLACEMENTS (same as STATA)
# ----------------------------------------------------
replace_map = {
    "Indiana University Bloomington": "IU Kelley School of Business",
    "Universite de Montreal": "HEC Montreal",
    "IESE Business School": "University of Navarra"
}
df_univ["university"] = df_univ["university"].replace(replace_map)

# ----------------------------------------------------
# DROP duplicates per paper (same as: by ut aff: keep if _n==_N)
# ----------------------------------------------------
df_univ = df_univ.drop_duplicates(subset=["paper_id", "university"], keep="last")

# ----------------------------------------------------
# FLAG: University Systems (univSystem == 1)
# ----------------------------------------------------
df_univ["univSystem"] = df_univ["university"].str.contains("System", case=False, regex=False)

# ----------------------------------------------------
# FLAG: Duplicated names requiring removal (duplic==1)
# ----------------------------------------------------
duplic_patterns = [
    "University of London",
    "University of North Carolina",
    "University of Washington",
    "Pennsylvania State University",
    "Arizona State University",
    "Indiana University Bloomington",
    "Nanyang Technological University & National Institute of Education (NIE)",
    "IESE Business School",
    "University of Illinois Chicago Hospital",
    "Universite de Montreal",
    "University of South Carolina",
    "Mays Business School",
    "New York University Tandon School of Engineering",
    "University of North Carolina School of Medicine",
    "",
    "Erasmus University Rotterdam - Excl Erasmus MC"
]

pattern_regex = "|".join([f"^{p}$" if p else r"^$" for p in duplic_patterns])
df_univ["duplic"] = df_univ["university"].str.match(pattern_regex, case=False)

# ----------------------------------------------------
# FLAG: Centers (centers == 1)
# ----------------------------------------------------
center_patterns = [
    "National Bureau of Economic Research",
    "Federal Reserve Bank",
    "Centre for Economic Policy Research - UK",
    "Center for Economic & Policy Research (CEPR)",
    "UDICE-French Research Universities - investigate",
    "Centre National de la Recherche Scientifique (CNRS)",
    "Chinese Academy of Sciences",
    "European Corporate Governance Institute",
    "Swiss Federal Institutes of Technology Domain"
]

df_univ["centers"] = df_univ["university"].apply(
    lambda x: any(p.lower() in x.lower() for p in center_patterns)
)

# ----------------------------------------------------
# APPLY STATA DROPS
# ----------------------------------------------------
df_univ = df_univ[df_univ["univSystem"] != True]
df_univ = df_univ[df_univ["duplic"] != True]
df_univ = df_univ[df_univ["centers"] != True]
df_univ = df_univ[df_univ["university"] != "Microsoft"]

# --------------------------------------------------------
# LIST OF BANK / CENTRAL BANK AFFILIATIONS TO REMOVE
# --------------------------------------------------------
BANK_NAMES = [
    "European Central Bank",
    "The World Bank",
    "Bank of Canada",
    "Bank of Italy",
    "Bank for International Settlements (BIS)",
    "Deutsche Bundesbank",
    "Bank of England",
    "Bank of France",
    "Bank of Finland",
    "Egyptian Knowledge Bank (EKB)",
    "Inter-American Development Bank",
    "De Nederlandsche Bank NV",
    "Swiss National Bank (SNB)",
    "Norges Bank",
    "Sveriges Riksbank",
    "Central Bank of Chile",
    "National Bank of Slovakia",
    "Central Bank of Ireland",
    "Central Bank of Brazil",
    "European Bank of Reconstructon & Development",
    "Reserve Bank of India",
    "National Bank of Belgium",
    "Bank of Israel",
    "Bank of Mexico",
    "Bank of Lithuania",
    "Bank of America Corporation",
    "National Bank of Ukraine",
    "Bank of Japan",
    "African Development Bank Group (AfDB)",
    "Industrial & Commercial Bank of China (ICBC)",
    "London South Bank University",
    "University of Alaska Fairbanks",
    "Banking Academy of Vietnam",
    "Central Bank of the Republic of Turkey",
    "Commerzbank AG",
    "Bank of Communications",
    "Yapi Kredi Bank",
    "Oesterreichische Nationalbank (OeNB)",
    "People's Bank of China",
    "Bank of Korea"
]

# --------------------------------------------------------
# REMOVE BANKS / CENTRAL BANKS FROM UNIVERSITY LIST
# --------------------------------------------------------
df_univ = df_univ[~df_univ["university"].isin(BANK_NAMES)]

# ----------------------------------------------------
# DROP helper columns
# ----------------------------------------------------
df_univ = df_univ.drop(columns=["univSystem", "duplic", "centers"])

# ----------------------------------------------------
# LONG FORMAT: Paper × Author
# ----------------------------------------------------
rows_auth = []

for idx, row in df.iterrows():
    for author in row["author_list"]:
        rows_auth.append({
            "paper_id": int(idx),
            "title": row["title"],
            "year": int(row["year"]),
            "journal": row["journalFullName"],
            "author": author,
            "disciplineAbbr": row["disciplineAbbr"],
            "utd24": int(row["utd24"]),
            "ft50": int(row["ft50"])
        })

df_auth = pd.DataFrame(rows_auth)

In [2]:
# -----------------------------
# LOAD DATA
# -----------------------------

lookup = pd.read_stata("authorsLookUpFT50_30Oct2025_v1gz.dta")

# Make sure columns we use exist:
# df columns: paper_id, title, year, journal, author, disciplineAbbr, utd24, ft50
# lookup columns: authnm, authnm2 (check this!)
# rename `author` → `authnm` to follow STATA flow
df_auth = df_auth.rename(columns={"author": "authnm"})


# ======================================================
# STATA: replace authnm = subinstr(authnm, " ", "", 1) 
#        if substr(authnm,1,1) == " "
# Meaning: Remove ONE leading space
# ======================================================

df_auth["authnm"] = df_auth["authnm"].apply(lambda x: x[1:] if isinstance(x, str) and x.startswith(" ") else x)

# ======================================================
# STATA: drop if authnm == ""
# ======================================================

df_auth = df_auth[df_auth["authnm"].astype(str).str.strip() != ""]


# ======================================================
# STATA: by ut authnm, sort: keep if _n == _N
# ut = unique paper_id
# Keep last duplicate per (paper_id, authnm)
# ======================================================

df_auth = df_auth.sort_values(["paper_id", "authnm"])
df_auth = df_auth.drop_duplicates(subset=["paper_id", "authnm"], keep="last")


# ======================================================
# STATA: merge m:1 authnm using lookup
# ======================================================

lookup = lookup.rename(columns=str.lower)   # normalize names
df_auth["authnm"] = df_auth["authnm"].astype(str)

merged = df_auth.merge(lookup, on="authnm", how="left", indicator=True)

# STATA: drop if _merge == 2 (lookup-only rows)
merged = merged[merged["_merge"] != "right_only"]

# STATA: replace authnm2 = authnm if authnm2 == ""
merged["authnm2"] = merged["authnm2"].fillna("")
mask_empty = merged["authnm2"].astype(str).str.strip() == ""
merged.loc[mask_empty, "authnm2"] = merged.loc[mask_empty, "authnm"]


# ======================================================
# Special substitutions (STATA)
# ======================================================

merged["authnm2"] = merged["authnm2"].replace({
    "Gupt, Alok": "Gupta, Alok",
    "Tucker, Catherine Elizabeth": "Tucker, Catherine E."
})


# ======================================================
# STATA: drop if authnm2 == ""
# ======================================================

merged = merged[merged["authnm2"].astype(str).str.strip() != ""]


# ======================================================
# STATA: drop if authnm2 == "[Anonymous]"
# ======================================================

merged = merged[merged["authnm2"] != "[Anonymous]"]


# ======================================================
# STATA: by ut authnm2, sort: keep if _n == _N
# (remove duplicate authors on the same paper)
# ======================================================

merged = merged.sort_values(["paper_id", "authnm2"])
merged = merged.drop_duplicates(subset=["paper_id", "authnm2"], keep="last")


# ======================================================
# Rename cleaned author name back to "author" 
# ======================================================

merged = merged.rename(columns={"authnm2": "author"})


# ======================================================
# EXPORT CLEANED FILE
# ======================================================
merged = merged.drop(columns=["authnm", "_merge"])
merged.to_csv("paper_author_long_CLEANED.csv", index=False)
df_auth = merged
print("Cleaning complete. Final rows:", len(merged))

Cleaning complete. Final rows: 171697


In [3]:
df_univ.to_csv("paper_university_long.csv", index=False)
df_auth.to_csv("paper_author_long.csv", index=False)

In [4]:
df_univ.to_json("universities.json", orient="records")
df_auth.to_json("authors.json", orient="records")

In [5]:
#subset_univ = df_univ[df_univ["year"] >= 2016]
#subset_auth = df_auth[df_auth["year"] >= 2016]

In [6]:
#subset_univ.to_json("universitiesSub.json", orient="records")
#subset_auth.to_json("authorsSub.json", orient="records")

In [7]:
import pandas as pd

# Load Excel
df = pd.read_excel("51journals_UTD_FT_openalexWOSdoi_utdft.xlsx")

# Standardize column names
df.columns = df.columns.str.strip().str.lower()

# Rename columns if needed
rename_map = {
    "journal_fullname": "journal",
    "journalfullname": "journal",
    "journal_full_name": "journal",
    "disciplineabbr": "disciplineabbr",
}
df = df.rename(columns=rename_map)

# Keep only necessary columns
df = df[
    ["title", "year", "journal", "authors", "affiliations",
     "disciplineabbr", "utd24", "ft50"]
]

# Filter years
#df = df[(df["year"] >= 2016) & (df["year"] <= 2024)]

# Drop rows missing essentials
df = df.dropna(subset=["title", "year", "journal"]).reset_index(drop=True)

# ✅ THIS IS THE KEY LINE — NaN → null automatically
#df.to_json("articlesSub.json", orient="records")
df.to_json("articles.json", orient="records")

print("✔ Saved valid JSON: articlesSub.json")


✔ Saved valid JSON: articlesSub.json
